<a href="https://colab.research.google.com/github/minkyoJang/AI-BigDataStudy/blob/master/jmkDay23_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/drive/')
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy

In [0]:
from konlpy.tag import Okt
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import numpy as np
import pickle
import torch

okt = Okt()

In [0]:
!ls "/drive/My Drive/lectures/deep_learning/2강/data"

data_deep.pkl  data_ml.pkl


In [0]:
with open('/drive/My Drive/lectures/deep_learning/2강/data/data_deep.pkl', 'rb') as f:
    index2voca = pickle.load(f)
    voca2index = pickle.load(f)
    train_X = pickle.load(f)
    train_y = pickle.load(f)
    test_X = pickle.load(f)
    test_y = pickle.load(f)

In [0]:
train_X, train_y = torch.LongTensor(train_X), torch.FloatTensor(train_y)
test_X, test_y = torch.LongTensor(test_X), torch.FloatTensor(test_y)

train_y, test_y = train_y.view(-1, 1), test_y.view(-1, 1)

train_dataset = TensorDataset(train_X, train_y)
test_dataset = TensorDataset(test_X, test_y)

train_loader = DataLoader(train_dataset, batch_size=128)
test_loader = DataLoader(test_dataset, batch_size=128)

In [0]:
class CNNRegressor(nn.Module):
    def __init__(self, voca_num, embedding_dim, filter_lengths, filter_num=20):
        super(CNNRegressor, self).__init__()
        
        self.voca_num = voca_num
        self.embedding_dim = embedding_dim
        self.filter_lengths = filter_lengths
        self.filter_num = filter_num
        
        self.embedding = nn.Embedding(self.voca_num, self.embedding_dim, padding_idx=0)
        self.filters = nn.ModuleList([nn.Conv1d(1, self.filter_num, (l, self.embedding_dim)) \
                                      for l in self.filter_lengths])
        
        self.linear1 = nn.Linear(self.filter_num*len(self.filter_lengths), 10)
        self.linear2 = nn.Linear(10, 1)
        
    def forward(self, words):
        embs = self.embedding(words)
        embs = embs.unsqueeze(1)
        features = [F.relu(conv(embs)) for conv in self.filters]
        features = [f.squeeze(3) for f in features]
        features = [F.max_pool1d(f, f.size(2)) for f in features]
        output = torch.cat(features, dim=1)
        output = output.squeeze(2)
        output = torch.relu(self.linear1(output))
        output = self.linear2(output)
        return output
    

class RNNRegressor(nn.Module):
    def __init__(self, voca_num, embedding_dim, hidden_dim, num_layer=2, bidirectional=True):
        super(RNNRegressor, self).__init__()
        self.voca_num = voca_num
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layer = num_layer
        self.bidirectional = bidirectional
        
        self.embedding = nn.Embedding(voca_num, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=self.num_layer,
                          bidirectional=self.bidirectional, batch_first=True)
        self.linear1 = nn.Linear(hidden_dim*(self.bidirectional + 1), 10)
        self.linear2 = nn.Linear(10, 1)
        
    def forward(self, words):
        embs = self.embedding(words)
        output, h_n = self.gru(embs)
        output = output[:, -1, :]
        output = torch.relu(self.linear1(output))
        output = self.linear2(output)
        return output

In [0]:
device = torch.device('cuda')

# reg = CNNRegressor(len(index2voca), 128, [2, 3, 4])
reg = RNNRegressor(len(index2voca), 256, 256, num_layer=2, bidirectional=True)
reg = reg.to(device)

criterion = nn.MSELoss()
optim = torch.optim.Adam(reg.parameters(), lr=1e-3, weight_decay=1e-6)

In [0]:
epochs = 10

reg.train()
for e in range(epochs):
    for i, (batch_X, batch_y) in enumerate(train_loader):
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        predict = reg(batch_X)
        
        loss = criterion(predict, batch_y)
        optim.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(reg.parameters(), 0.5)
        optim.step()
        
        if i % 100 == 0:
            loss = loss.item()
            print(f"{e}epochs, {i}iters - {loss}")

In [0]:
total_loss = []
test_num = 0
l1_loss = nn.L1Loss()

with torch.no_grad():
    reg.eval()
    for i, (batch_X, batch_y) in enumerate(test_loader):
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        predict = reg(batch_X)
        predict = torch.clamp(predict, min=0, max=1)
        
        loss = l1_loss(predict, batch_y)
        loss = loss.item()
        batch_size = batch_X.size(0)
        test_num += batch_size
        total_loss.append(loss*batch_size)
        
total_loss = np.sum(total_loss)/test_num
print(total_loss)

0.2779954965793842


In [0]:
text = "이 영화 정말 감동적이다"
parsed = okt.morphs(text)

vector = np.zeros((1, train_X.shape[1]))
for i, w in enumerate(parsed):
    if w in voca2index:
        vector[0, i] += voca2index[w]
    
vector = torch.from_numpy(vector).long().to(device)
predict = torch.sigmoid(reg(vector)).item()
print(predict)

0.6258990168571472
